In [6]:
import gradio as gr
import psycopg2
import json
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

#local import 
from search import search


def search_html(query):
    results = search(query,model, db_config)
    # Format the results as HTML-formatted text
    html_output = ""
    for project_id, chunk, distance in results:
        project = next((p for p in projects if p["ids"] == [project_id]), None)
        html_output += f"<h3>Project ID: {project_id}</h3>"
        html_output += f"<h4>Project title: {project['title']}</h4>"
        html_output += f"<p><b>Project abstract:</b> {project['abstract']}</p>"
        html_output += f"<p><b>Relevant snippet:</b> {chunk}</p>"
        html_output += f"<p><b>Distance:</b> {distance}</p>"
        html_output += "<hr>"
    return html_output


number_of_results = 5

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Define Gradio input and output interfaces
input_interface = gr.inputs.Textbox(lines=3, label="Enter your query")
output_interface = gr.outputs.HTML(label="Search Results")

# Create the Gradio interface
iface = gr.Interface(
    fn=search_html,
    inputs=input_interface,
    outputs=output_interface,
    title="Project Search",
    description="Search for relevant projects using natural language queries.",
)

# Launch the Gradio interface
iface.launch()


/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Project ID: P151072
Project title: India - National Agriculture Higher Education Project : indigenous peoples plan : Indigenous peoples policy framework
Project abstract: The objective of the National Agriculture Higher Education Project for India is: ‘to support participating Agricultural Universities and ICAR in providing more relevant and higher quality education to agriculture university students.’ ICAR has prepared this Equity Action Plan (EAP) or Indigenous People’s Policy Framework (IPPF) which addresses issues of gender equality and social inclusion with special attention to the needs of the Scheduled Tribe and the Scheduled Caste students and faculty members fulfilling the requirements of OP 4.10. This EAP/IPPF has been prepared using mostly qualitative research methodologies, including an online survey with the primary stakeholder - students and faculties from various social backgrounds, including ST and SC groups. This will be discussed and finalized with intensive stakehold

Traceback (most recent call last):
  File "/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/routes.py", line 393, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/blocks.py", line 1069, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/gradio/blocks.py", line 878, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brunosan/opt/miniconda3/envs/s2/lib/python3.11/site-packages/anyio/_backends/